In [2]:

import pandas as pd
import numpy as np
import math
from collections import Counter

In [3]:
sensor_data = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
sensor_data_acc = sensor_data[sensor_data.SENSORTYPE == 1]
sensor_data_acc_tag12 = sensor_data_acc[(sensor_data_acc.TagName == 'Tag1') | (sensor_data_acc.TagName == 'Tag2')]

In [131]:
tag_id_groupby = sensor_data_acc_tag12.groupby(['TagName', 'tester_id'])
tag_id_dict = tag_id_groupby.groups
y = list(tag_id_dict.keys())
y = [i[0] for i in y]
y = np.array(y)

In [107]:
# there will be 9 frames, frame0 to frame8
# There will be 10 segments
N_frame_no = 9 

test_percent = 0.1 # 10% samples are used for testing

In [92]:
def gesture_features(accs):
    Ls = math.floor(len(accs)/ (N_frame_no + 1))
    segments = None
    for i in range(N_frame_no + 1):
        if segments is None:
            segments = np.array([accs[i*Ls:(i+1)*Ls]])
        else:
            segments = np.append(segments, np.array([accs[i*Ls:(i+1)*Ls]]), axis=0)

    frames = None
    for i in range(N_frame_no):
        cur_frame = segments[i:i+2]
        cur_frame = cur_frame.reshape((cur_frame.shape[0]*cur_frame.shape[1],cur_frame.shape[2]))
        if frames is None:
            frames = np.array([cur_frame])
        else:
            frames = np.append(frames, np.array([cur_frame]), axis = 0)
    return np.array([frame_features(f,Ls) for f in frames]).reshape(-1)

In [88]:
def frame_features(cur_frame, Ls):
    dft_cur_frame = np.fft.fftn(cur_frame)
    
    mean_cur_frame = dft_cur_frame[0]

    energy_cur_frame=[]
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        for i in range(1,Ls*2):
            T_sum += math.pow(abs(dft_cur_frame[i,T]),2)

        energy_cur_frame.append(T_sum / abs(Ls*2-1))
    energy_cur_frame = np.array(energy_cur_frame)
    
    entropy_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        entropy_sum = 0
        for i in range(1,Ls*2):
            T_sum += abs(dft_cur_frame[i,T])

        for m in range(1,Ls*2):
            p_m_T = abs(dft_cur_frame[m,T])/T_sum
            entropy_sum += p_m_T*math.log(1/p_m_T)
        entropy_cur_frame.append(entropy_sum)
    entropy_cur_frame = np.array(entropy_cur_frame)
    
    std_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        std_cur_frame.append(np.std(cur_frame))
    std_cur_frame = np.array(std_cur_frame)
    
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    ## TODO     ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO    ## TODO
    # correlation
    
    return np.concatenate((mean_cur_frame,energy_cur_frame,std_cur_frame,entropy_cur_frame))

array([  1.82634742e+02  +0.j        ,  -7.23970814e+02+504.97138681j,
        -7.23970814e+02-504.97138681j,   6.64357479e-02  +0.j        ,
         1.27755739e+00  +0.j        ,   9.32119271e-01  +0.j        ,
         5.33619203e+00  +0.j        ,   5.33619203e+00  +0.j        ,
         5.33619203e+00  +0.j        ,   1.94913220e+01  +0.j        ,
         1.50624683e+01  +0.j        ,   1.55994969e+01  +0.j        ])

In [95]:
X = []
for key in list(tag_id_dict.keys()):
    X.append(gesture_features(tag_id_groupby.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values).reshape(-1))
X = np.array(X)

In [96]:
X.shape

(214, 108)

In [136]:
from sklearn.svm import SVC
clf = SVC(kernel='poly', C=1)

In [137]:
idx_list = list(range(len(X)))
np.random.shuffle(idx_list)
train_idx = idx_list[:math.floor(len(X) * (1- test_percent))]
test_idx = idx_list[math.floor(len(X) * (1- test_percent)):]

In [138]:
# y[y == 'Tag2'] = 2
# y[y == 'Tag1'] = 1

train_x = X[train_idx]
train_y = y[train_idx]
test_x = X[test_idx]
test_y = y[test_idx]

In [139]:
clf.fit(train_x, train_y) 

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


array(['Tag1', 'Tag2', 'Tag2', 'Tag2', 'Tag1', 'Tag1', 'Tag2', 'Tag1',
       'Tag1', 'Tag1', 'Tag1', 'Tag1', 'Tag1', 'Tag2', 'Tag2', 'Tag1',
       'Tag2', 'Tag1', 'Tag1', 'Tag1', 'Tag2', 'Tag2'],
      dtype='<U4')

In [142]:
accuracy_score(test_y, clf.predict(test_x))

NameError: name 'accuracy_score' is not defined